In [ ]:
'''
Automating 10fastfingers website using selenium
Part 1: Login
'''
user_email = "abc@mail.com"
user_password = "1234"

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import cv2
import pytesseract
from time import sleep
from PIL import Image

DRIVER_LOCATION = "/snap/bin/chromium.chromedriver"
BINARY_LOCATION = "/usr/bin/chromium-browser"
option = Options()
option.binary_location = BINARY_LOCATION

driver = webdriver.Chrome(DRIVER_LOCATION, options=option)
driver.get("https://10fastfingers.com/login")


load_time = 20

email_in = WebDriverWait(driver, load_time).until(
            EC.presence_of_element_located((By.ID, "UserEmail"))
)

email_in.send_keys(user_email)

pass_in = WebDriverWait(driver, load_time).until(
            EC.presence_of_element_located((By.ID, "UserPassword"))
)
pass_in.send_keys(user_password)
pass_in.send_keys(Keys.RETURN)


In [ ]:
'''
Part 2: Automating the type test.
'''

rem = 0
while rem < 10:
    div_father = WebDriverWait(driver, load_time).until(
            EC.presence_of_element_located((By.ID, "row1"))
    )
    words = []
    for span in div_father.find_elements_by_tag_name("span"):
        
        if span.text != "" and span.get_attribute("class") != "correct":
            words.append(span.text)
        else: continue

    inputfield = WebDriverWait(driver, load_time).until(
            EC.presence_of_element_located((By.ID, "inputfield"))
    )

    for word in words:
        inputfield.send_keys(word)
        inputfield.send_keys(Keys.SPACE)
    rem += 1

In [ ]:
'''
Part 3: proving we are not bot :|   (have no idea how it works, to be honest)
'''

start_button = WebDriverWait(driver, load_time).until(
    EC.presence_of_element_located((By.ID, "start-btn"))
)
start_button.click()
sleep(2)

div_img = WebDriverWait(driver, load_time).until(
    EC.presence_of_element_located((By.ID, "word-img"))
)

element = div_img.find_element_by_tag_name("img")
location = element.location
size = element.size
driver.save_screenshot("pageImage.png")

# crop image
x = location['x']
y = location['y']
width = location['x']+size['width']
height = location['y']+size['height']
im = Image.open('pageImage.png')
im = im.crop((int(x), int(y), int(width), int(height)))
im.save('text.png')


img = cv2.imread("text.png")


# Convert the image to gray scale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Performing OTSU threshold
ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)


rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))

dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)

contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
												cv2.CHAIN_APPROX_NONE)

# Creating a copy of image
im2 = img.copy()

answer = ""

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    
    # Drawing a rectangle on copied image
    rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    cropped = im2[y:y + h, x:x + w]
    
    text = pytesseract.image_to_string(cropped)
    
    answer += text + " "
    
word_input = WebDriverWait(driver, load_time).until(
    EC.presence_of_element_located((By.ID, "word-input"))
)
start_number = 0
answer = answer.replace("\n", " ")

word_input.send_keys(answer.strip())